In [31]:
import requests
import json

url = 'https://api.opendota.com/api/publicMatches'
params = {
    'less_than_match_id': 7045890001,
    'limit': 100,
    'api_key': '4dad5ea5-a373-4980-9ecb-20552c4e2ccd'
}

matches = []

for i in range(30):
    print(f'Retrieving matches for minute {i+1}...')
    start_time = time.time()
    
    for j in range(1200):
        response = requests.get(url, params=params)
        
        # Check if status code is 200 OK
        if response.status_code == requests.codes.ok:
            response_json = json.loads(response.text)
            
            # Check if response JSON is a list of dictionaries (for parsing and progress-check purpose)
            if isinstance(response_json, list) and all(isinstance(match, dict) for match in response_json):
                matches.extend(response_json)
            else:
                print('Error: invalid response format')
                break
        else:
            print(f'Error: {response.status_code}')
            break
        
        if len(matches) >= 100000:
            break
        
        last_match_id = response_json[-1]['match_id']
        params['less_than_match_id'] = last_match_id
    
    elapsed_time = time.time() - start_time
    print(f'Retrieved {len(matches)} matches in {elapsed_time:.2f} seconds')
    print('-' * 50)
    time.sleep(max(0, 60 - elapsed_time))

Retrieving matches for minute 1...
Retrieved 100000 matches in 143.58 seconds
--------------------------------------------------
Retrieving matches for minute 2...
Retrieved 100100 matches in 0.12 seconds
--------------------------------------------------
Retrieving matches for minute 3...
Retrieved 100200 matches in 0.12 seconds
--------------------------------------------------
Retrieving matches for minute 4...
Retrieved 100300 matches in 0.15 seconds
--------------------------------------------------
Retrieving matches for minute 5...
Retrieved 100400 matches in 0.12 seconds
--------------------------------------------------


KeyboardInterrupt: 

In [39]:
import pandas as pd
#to dataframe
a = pd.DataFrame.from_dict(matches)
a

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,7045889918,5915914855,False,1678031918,2199,2607.0,2.0,7,3,31,5,227,"9,70,50,52,136","69,31,27,40,8"
1,7045889916,5915908457,True,1678031918,1920,NaN,NaN,7,22,35,4,185,"14,87,40,73,56","101,107,77,111,93"
2,7045889915,5915916353,True,1678031918,2301,2931.0,1.0,0,22,41,4,191,"44,81,101,50,86","27,6,23,14,25"
3,7045889914,5915936872,True,1678031917,3131,NaN,NaN,7,22,13,2,183,"21,104,120,67,1","80,128,35,63,5"
4,7045889913,5915912349,True,1678031917,2093,1942.0,1.0,7,22,22,5,202,"41,62,31,17,14","104,15,54,112,2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100395,7044134811,5914342148,False,1677947491,1821,4004.0,2.0,7,22,44,5,203,"25,100,135,53,9","97,40,26,56,54"
100396,7044134808,5914365565,False,1677947480,2726,NaN,NaN,7,22,41,3,251,"86,25,98,19,22","53,64,30,54,62"
100397,7044134807,5914341797,True,1677947461,1803,2416.0,3.0,0,22,43,4,122,"75,26,14,104,35","120,74,83,67,19"
100398,7044134715,5914381909,True,1677947495,3309,3740.0,1.0,7,22,61,2,274,"43,41,26,30,123","89,11,100,63,14"


In [46]:
#check if there is 100k unique matches
num_unique_matches = a['match_id'].nunique()
num_unique_matches

100000

In [47]:
#drop unnecessary or unused columns
a = a.drop(columns=['match_seq_num', 'start_time', 'num_mmr', 'lobby_type', 'game_mode', 'num_rank_tier', 'cluster'])
a

,match_id,radiant_win,duration,avg_mmr,avg_rank_tier,radiant_team,dire_team
0,7045889918,False,2199,2607.0,31,"9,70,50,52,136","69,31,27,40,8"
1,7045889916,True,1920,NaN,35,"14,87,40,73,56","101,107,77,111,93"
2,7045889915,True,2301,2931.0,41,"44,81,101,50,86","27,6,23,14,25"
3,7045889914,True,3131,NaN,13,"21,104,120,67,1","80,128,35,63,5"
4,7045889913,True,2093,1942.0,22,"41,62,31,17,14","104,15,54,112,2"
...,...,...,...,...,...,...,...
100395,7044134811,False,1821,4004.0,44,"25,100,135,53,9","97,40,26,56,54"
100396,7044134808,False,2726,NaN,41,"86,25,98,19,22","53,64,30,54,62"
100397,7044134807,True,1803,2416.0,43,"75,26,14,104,35","120,74,83,67,19"
100398,7044134715,True,3309,3740.0,61,"43,41,26,30,123","89,11,100,63,14"


In [48]:
#write csv
a.to_csv('matches.csv', index=False)